## Notebook

Add julia functions by calling

In [25]:
include("Julia_functions/solution_animation.jl")
include("Julia_functions/indicator_chi.jl")


χ (generic function with 1 method)

In a julia code env

In [2]:
using Gridap, GridapMakie, CairoMakie, FileIO

#GLMakie

In [ ]:
domain = (-1, 1, -1, 1)
partition = nx, ny = 10,10
model = CartesianDiscreteModel(domain, partition) |> simplexify

Ω = Triangulation(model)                                # make Triangulation
# fig = plot(Ω)
# wireframe!(Ω, color=:black)                             # Plot edges of each element
# scatter!(Ω, marker=:circle, markersize=20, color=:blue) # Plot mesh nodes
# display(fig)                                            # show plot


order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1)     # no Dirichlet (/ everything Neumann)

Trialspace = TrialFESpace(Testspace)                    # make trial space based on test space

degree = 2*order                                        # degree of the method used for approximating integrals over Ω                   
dΩ = Measure(Ω,degree)                                  # make the measure dΩ

Γ = BoundaryTriangulation(model)                        # triangulate the boundary ∂Ω
dΓ = Measure(Γ,degree)                                  # measure on Γ


# Varmekilden q er 1 i dette området og 0 ellers
Q(t) = χ(t, 0, 0.03)*100
q_pos(x) = χ(x[1], -0.05, 0.05) * χ(x[2], -0.10, 0.10)


τ = 0.01
k = 1
h = 0.01
T_old = 290
T_out = 290


# To store the solution over time
solutions = []

for step in 1:100
    
    a(T,ϕ) = ∫( T/τ*ϕ + k*∇(ϕ)⊙∇(T))*dΩ - ∫(k*h*T*ϕ)*dΓ
    l(ϕ) = ∫(q_pos*ϕ*Q(step*τ) + T_old/τ*ϕ)*dΩ - ∫( k*h*T_out*ϕ)*dΓ 
    
    # Set up system matrix
    op = AffineFEOperator(a,l,Trialspace,Testspace)

    # Solve system matrix
    uh = solve(op) 
    
    # Store the solution
    push!(solutions, uh)

    # Update T_old
    T_old = uh
end

solution_animation(solutions, 30)


"animation.mp4"